In [1]:
# 모듈 로딩
# -Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F # 변수명과 헷갈릴까봐 소문자는 사용 X

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchmetrics.classification import F1Score, BinaryF1Score
from torchmetrics.classification import BinaryConfusionMatrix

from torchinfo import summary

# - Data 및 시각화 관련

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split 


#사용자 정의 함수
from  myfunc import *
from my_module import SmokingBCFModel
from my_module import SmokingDataset
from my_module import DynamicModel
from preprocessing_func import *

In [2]:
DATA_FILE='diabetes_binary_health_indicators_BRFSS2021.csv'

DF=pd.read_csv(DATA_FILE)

DF.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0,1.0,1,15.0,1.0,0.0,0.0,0,1,...,1,0.0,5.0,10.0,20.0,0.0,0,11,4.0,5.0
1,1.0,1,0.0,1,28.0,0.0,0.0,1.0,0,1,...,1,0.0,2.0,0.0,0.0,0.0,0,11,4.0,3.0
2,1.0,1,1.0,1,33.0,0.0,0.0,0.0,1,1,...,1,0.0,2.0,10.0,0.0,0.0,0,9,4.0,7.0
3,1.0,0,1.0,1,29.0,0.0,1.0,1.0,1,1,...,1,0.0,5.0,0.0,30.0,1.0,1,12,3.0,4.0
4,0.0,0,0.0,1,24.0,1.0,0.0,0.0,0,0,...,1,0.0,3.0,0.0,0.0,1.0,1,13,5.0,6.0


In [3]:
DF=preprocess_and_encode(DF)

In [4]:
DF

,Diabetes_binary,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,BMI_비만 3단계,BMI_초고도비만,BMI_극단적 비만,GenHlth_1-3,GenHlth_4-5,Age_1-8,Age_9-13,Income_1-5,Income_6-8,Income_9-11
0,0.0,0,1.0,1,1.0,0.0,0.0,0,1,1,...,0,0,0,0,1,0,1,1,0,0
1,1.0,1,0.0,1,0.0,0.0,1.0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
2,1.0,1,1.0,1,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0,0,1,0,1,0
3,1.0,0,1.0,1,0.0,1.0,1.0,1,1,1,...,0,0,0,0,1,0,1,1,0,0
4,0.0,0,0.0,1,1.0,0.0,0.0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236373,1.0,1,1.0,1,0.0,0.0,0.0,1,1,1,...,0,0,0,0,1,0,1,1,0,0
236374,0.0,1,0.0,1,1.0,0.0,0.0,1,1,1,...,0,0,0,1,0,1,0,1,0,0
236375,0.0,0,1.0,1,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0,1,0,0,0,1
236376,0.0,1,0.0,1,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0,0,1,0,1,0


In [5]:
# - DataFrame에서 피쳐와 타겟 추출

targetDF = DF['Smoker']  .to_frame(name='Smoker')    # 2D (150, 1)
featureDF = DF.drop('Smoker',axis=1)    # 2D (150, 3)

In [6]:
targetDF

,Smoker
0,1.0
1,0.0
2,0.0
3,0.0
4,1.0
...,...
236373,0.0
236374,1.0
236375,0.0
236376,0.0


In [7]:
featureDF

,Diabetes_binary,HighBP,HighChol,CholCheck,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,...,BMI_비만 3단계,BMI_초고도비만,BMI_극단적 비만,GenHlth_1-3,GenHlth_4-5,Age_1-8,Age_9-13,Income_1-5,Income_6-8,Income_9-11
0,0.0,0,1.0,1,0.0,0.0,0,1,1,0,...,0,0,0,0,1,0,1,1,0,0
1,1.0,1,0.0,1,0.0,1.0,0,1,0,0,...,0,0,0,1,0,0,1,1,0,0
2,1.0,1,1.0,1,0.0,0.0,1,1,1,0,...,0,0,0,1,0,0,1,0,1,0
3,1.0,0,1.0,1,1.0,1.0,1,1,1,0,...,0,0,0,0,1,0,1,1,0,0
4,0.0,0,0.0,1,0.0,0.0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236373,1.0,1,1.0,1,0.0,0.0,1,1,1,0,...,0,0,0,0,1,0,1,1,0,0
236374,0.0,1,0.0,1,0.0,0.0,1,1,1,0,...,0,0,0,1,0,1,0,1,0,0
236375,0.0,0,1.0,1,0.0,0.0,1,1,1,0,...,0,0,0,1,0,1,0,0,0,1
236376,0.0,1,0.0,1,0.0,0.0,1,1,1,0,...,0,0,0,1,0,0,1,0,1,0


In [8]:
### [테스트] 모델 인스턴스 생성
model = SmokingBCFModel()

print(model)

SmokingBCFModel(
  (in_layer): Linear(in_features=29, out_features=10, bias=True)
  (hd_layer1): Linear(in_features=10, out_features=30, bias=True)
  (hd_layer2): Linear(in_features=30, out_features=60, bias=True)
  (hd_layer3): Linear(in_features=60, out_features=30, bias=True)
  (hd_layer4): Linear(in_features=30, out_features=10, bias=True)
  (hd_layer5): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [9]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model,input_size=(2000,29))

Layer (type:depth-idx)                   Output Shape              Param #
SmokingBCFModel                          [2000, 1]                 --
├─Linear: 1-1                            [2000, 10]                300
├─Linear: 1-2                            [2000, 30]                330
├─Linear: 1-3                            [2000, 60]                1,860
├─Linear: 1-4                            [2000, 30]                1,830
├─Linear: 1-5                            [2000, 10]                310
├─Linear: 1-6                            [2000, 5]                 55
├─Linear: 1-7                            [2000, 1]                 6
Total params: 4,691
Trainable params: 4,691
Non-trainable params: 0
Total mult-adds (M): 9.38
Input size (MB): 0.23
Forward/backward pass size (MB): 2.34
Params size (MB): 0.02
Estimated Total Size (MB): 2.59

In [10]:
## [테스트] 데이터셋 인스턴스 생성

# - 커스텀데이터셋 인스턴스 생성
SmokingDS = SmokingDataset(featureDF, targetDF)

# - 데이터로더 인스턴스 생성
irisDL = DataLoader(SmokingDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 29]) torch.Size([1, 1]) tensor([[0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.]]) tensor([[1.]])


In [17]:
### 학습 진행 관련 설정
EPOCH = 15

BATCH_SIZE = 1800 # 전체 데이터 수 확인후 지정 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.05  # 하이퍼파라미터, 값이 적을수록 촘촘하게 내려감

In [18]:
#  모델 인스턴스
model=SmokingBCFModel()

In [19]:
# DS과 DL인스턴스
# - 학습용, 검증용, 테스트용 데이터분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)

print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{y_train.value_counts()/y_train.shape[0]}{y_test.value_counts()/y_test.shape[0]}{y_val.value_counts()/y_val.shape[0]}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

# 학습용, 검증용, 테스트용 데이터셋
trainDS=SmokingDataset(X_train,y_train)
valDS=SmokingDataset(X_val,y_val)
testDS=SmokingDataset(X_test,y_test)


# - 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

(132962, 29) (59095, 29) (44321, 29)
(132962, 1) (59095, 1) (44321, 1)
Smoker
0.0       0.587739
1.0       0.412261
Name: count, dtype: float64Smoker
0.0       0.586006
1.0       0.413994
Name: count, dtype: float64Smoker
0.0       0.591458
1.0       0.408542
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [20]:
# 최적화 인스턴스 => w,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 분류 => 이진분류 BinaryCrossEntropyLoss => BCELoss
#                             예측값은 확률값으로 전달 ==> sigmoid() AF 처리후 전달
reqLoss=nn.BCELoss()

In [21]:
len(trainDL), trainDL.__len__()

(74, 74)

In [22]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]
CNT = len(trainDL)
print(f'CNT =>  {CNT}')


for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0

    for featureTS, targetTS in trainDL:
        # 학습 진행 
        pre_y=model(featureTS)

        # 손실 계산
        loss = reqLoss(pre_y,targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        #score= F1Score(task='binary')(pre_y,targetTS)
        score = BinaryF1Score()(pre_y, targetTS)
        score_total += score.item() # 텐서이기 때문에 .item() 해줘야 정수가 들어감

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포트 당 겅증기능
    # 모델 검증 모드 설정
    model.eval()
    
    with torch.no_grad():

        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        #추론/평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqLoss(pre_val,val_targetTS)
        
        # 성능평가
        score_val=BinaryF1Score()(pre_val, val_targetTS)


    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)


    # 손실값과 성능평가값 저장
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)


    print(f' [{epoch}/{EPOCH}]\n [TRAIN] LOSS : {LOSS_HISTORY[0][-1]}, SCORE : {SCORE_HISTORY[0][-1]}')
    print(f' [VALID] LOSS : {LOSS_HISTORY[1][-1]}, SCORE : {SCORE_HISTORY[1][-1]}')

CNT =>  74
 [0/15]
 [TRAIN] LOSS : 0.6564320970225979, SCORE : 0.3416822537075023
 [VALID] LOSS : 0.6431061029434204, SCORE : 0.4354821741580963
 [1/15]
 [TRAIN] LOSS : 0.643021710015632, SCORE : 0.4230033521716659
 [VALID] LOSS : 0.6436095237731934, SCORE : 0.4497309625148773
 [2/15]
 [TRAIN] LOSS : 0.6431809124108907, SCORE : 0.4192338783193279
 [VALID] LOSS : 0.6424912214279175, SCORE : 0.4062545895576477
 [3/15]
 [TRAIN] LOSS : 0.642210407031549, SCORE : 0.42973917944205775
 [VALID] LOSS : 0.6420746445655823, SCORE : 0.44884419441223145
 [4/15]
 [TRAIN] LOSS : 0.641845367244772, SCORE : 0.43064191293072057
 [VALID] LOSS : 0.6419190764427185, SCORE : 0.44749486446380615
 [5/15]
 [TRAIN] LOSS : 0.6414751805163719, SCORE : 0.43643857538700104
 [VALID] LOSS : 0.6426858901977539, SCORE : 0.4419628977775574
 [6/15]
 [TRAIN] LOSS : 0.641462546748084, SCORE : 0.43168929340066137
 [VALID] LOSS : 0.6413576006889343, SCORE : 0.4502117335796356
 [7/15]
 [TRAIN] LOSS : 0.6406929517114485, SCORE